# Hands-on AWS - 4 - Video analytics

## Installing dependencies
Here we install relevant dependencies to run serverless framework.

In [ ]:
!pip install awscli --upgrade --user
!curl -L https://github.com/stedolan/jq/releases/download/jq-1.6/jq-linux64 -o ~/.local/bin/jq
!chmod +x ~/.local/bin/jq

## Setting AWS environmental variables
Here we set up AWS environmental variables so that we will be able to deploy to our AWS account. Please replace it with provided credentials.

In [ ]:
%env AWS_ACCESS_KEY_ID=<AWS_ACCESS_KEY_ID>
%env AWS_SECRET_ACCESS_KEY=<AWS_SECRET_ACCESS_KEY>
%env AWS_ACCOUNT_ID=<AWS_ACCOUNT_ID>

## Setting up identifier

Here we set up personal identifier. You will need to replace "FirstName LastName" with your first name and lastname.


In [ ]:
import hashlib
import datetime
res=hashlib.sha224(bytes("FirstName LastName"+str(datetime.datetime.now().timestamp()),'utf-8')).hexdigest()[:5]
%env HASH=$res
%env AWS_DEFAULT_REGION=us-west-1

# Create S3 bucket

In [ ]:
!aws s3api create-bucket --bucket $HASH --create-bucket-configuration LocationConstraint=us-west-1

To delete the bucket, run `!aws s3api delete-bucket --bucket $HASH`

# Lambda deployment - object recog

Let's deploy the object recognition lambda.  
First we have to create a role which can be assumed by the lambda function.

In [ ]:
raw_output = !aws iam create-role --role-name lambda-role-$HASH --assume-role-policy-document file://hands-on-aws-4-video-analytics/roles/lambda-role.json | jq -r '.Role.Arn'
LAMBDA_ROLE_ARN = raw_output[0]
%env LAMBDA_ROLE_ARN=$LAMBDA_ROLE_ARN

Delete role in case of typo: `!aws iam  delete-role --role-name lambda-role-$HASH`

Create a policy with the list of permissions for the lambda

In [ ]:
raw_output = !aws iam create-policy --policy-name lambda-policy-$HASH --policy-document file://hands-on-aws-4-video-analytics/roles/lambda-policy.json | jq -r '.Policy.Arn'
LAMBDA_POLICY_ARN = raw_output[0]
%env LAMBDA_POLICY_ARN=$LAMBDA_POLICY_ARN

Delete role in case of typo: `!aws iam  delete-policy --policy-arn $LAMBDA_POLICY_ARN`

Now, we will attach the permissions for the above role.

In [ ]:
!aws iam attach-role-policy --role-name lambda-role-$HASH --policy-arn $LAMBDA_POLICY_ARN

Detach in case of typo: `!aws iam detach-role-policy --role-name lambda-role-$HASH --policy-arn $LAMBDA_POLICY_ARN`

Create lambda function from ECR image

In [ ]:
!aws lambda create-function --function-name recog-$HASH \
                            --package-type Image \
                            --code ImageUri="705254273855.dkr.ecr.us-west-1.amazonaws.com/video-analytics-recog-aws:latest" \
                            --role $LAMBDA_ROLE_ARN \
                            --timeout 120 \
                            --memory-size 4096 \
                            --environment Variables="{BUCKET_NAME=$HASH}" \
                            --tracing-config Mode=Active \
                            --publish

Delete command in case of typo: `!aws lambda delete-function --function-name recog-$HASH`

# Deploy decoder and streaming lambdas

In [ ]:
!bash "hands-on-aws-4-video-analytics/deploy-decoder-streaming-Lambda.sh"

# Lambda invoke - streaming
Invoke Lambda using cli

In [ ]:
!aws lambda invoke --function-name streaming-$HASH \
                   --payload '{ "name": "'$HASH'", "TransferType": "S3" }' response.json

# Lambda response

In [ ]:
!cat ./response.json

# Clean up resources

In [ ]:
!bash "hands-on-aws-4-video-analytics/cleanup.sh"